In [1]:
data_path="/data/linqinhong/zindi/covid_19/data"

In [2]:
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedKFold

In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import os

In [4]:
train_data=pd.read_csv(os.path.join(data_path,"Train.csv"))
test_data=pd.read_csv(os.path.join(data_path,"Test.csv"))

In [5]:
train_data

,ID,text,target
0,train_0,The bitcoin halving is cancelled due to,1
1,train_1,MercyOfAllah In good times wrapped in its gran...,0
2,train_2,266 Days No Digital India No Murder of e learn...,1
3,train_3,India is likely to run out of the remaining RN...,1
4,train_4,In these tough times the best way to grow is t...,0
...,...,...,...
5282,train_6856,The spread of the novel among asylum seekers o...,1
5283,train_6857,Hundreds of Jewish patients are being treated...,1
5284,train_6858,Beats me Honestly most of the people I follow ...,0
5285,train_6859,Help us reach more people by donating and shar...,0


In [6]:
train_data["process_text"]=train_data["text"].str.lower()

In [7]:
from nltk.corpus import stopwords
stopwords=set(stopwords.words("english"))
def remove_stopwords(text):
    return " ".join([i for i in text.split() if i not in stopwords])

In [8]:
import contractions
train_data["process_text"]=train_data["process_text"].apply(contractions.fix)

In [9]:
train_data["process_text"]=train_data["process_text"].apply(remove_stopwords)

In [10]:
def convert_to_vec(data):
    return torch.tensor(np.eye(2)[data["labels"]]).reshape([1,-1])

In [11]:
#train_data["process_target"]=train_data["target"].apply(convert_to_vec)

In [12]:
train_data.rename({"target":"labels"},axis=1,inplace=True)

In [13]:
train_data.drop("ID",axis=1,inplace=True)

In [14]:
train_data.drop("text",axis=1,inplace=True)

In [15]:
train_data["process_text"]

0                           bitcoin halving cancelled due
1       mercyofallah good times wrapped granular detai...
2       266 days digital india murder e learning 2g on...
3       india likely run remaining rna kits essential ...
4       tough times best way grow learn case teach hel...
                              ...                        
5282    spread novel among asylum seekers adds pile al...
5283    hundreds jewish patients treated arab practiti...
5284    beats honestly people follow sports fans l sha...
5285    help us reach people donating sharing ramadan ...
5286    interest rate swap derivative pricing python h...
Name: process_text, Length: 5287, dtype: object

In [16]:
from transformers import RobertaTokenizer
tokenizer=RobertaTokenizer.from_pretrained("roberta-large")

In [17]:
from datasets import Dataset
train_dataset=Dataset.from_pandas(train_data)
test_dataset=Dataset.from_pandas(test_data)

In [18]:
test_dataset

Dataset({
    features: ['ID', 'text'],
    num_rows: 1962
})

In [19]:
def tokenize(text):
    return tokenizer(text["process_text"],truncation=True,add_special_tokens=False,return_attention_mask=True)

In [20]:
train_dataset=train_dataset.map(tokenize,batched=True)

  0%|          | 0/6 [00:00<?, ?ba/s]

In [21]:
train_dataset.column_names

['labels', 'process_text', 'input_ids', 'attention_mask']

In [22]:
train_dataset["input_ids"]

[[47197, 12193, 6645, 8102, 528],
 [2089,
  4469,
  1116,
  18897,
  205,
  498,
  8144,
  17227,
  8244,
  4617,
  1539,
  465,
  3099,
  1421,
  27352],
 [31353,
  360,
  1778,
  9473,
  493,
  1900,
  364,
  2239,
  132,
  571,
  804,
  265,
  7057,
  1236,
  28127,
  449],
 [2028, 493, 533, 422, 2405, 910, 2133, 14506, 4499, 3044, 65, 186, 213, 705],
 [90,
  4894,
  498,
  275,
  169,
  1733,
  1532,
  403,
  6396,
  244,
  82,
  1532,
  4686,
  1612,
  26363],
 [506,
  15247,
  1850,
  2455,
  893,
  146,
  292,
  36553,
  15790,
  228,
  914,
  244,
  472,
  10731,
  671,
  814],
 [4082,
  3697,
  1612,
  941,
  216,
  2128,
  86,
  1022,
  626,
  766,
  20335,
  6549,
  86,
  49339,
  1524,
  2749,
  29130,
  556,
  65,
  1946,
  2526,
  86,
  1035,
  377,
  124,
  1136],
 [1023, 761, 11962, 2128, 5278, 298],
 [9289, 1902, 474, 1138, 2008, 1032, 19166, 6357, 1744, 16759, 405],
 [32233,
  5228,
  562,
  13203,
  2111,
  1142,
  31399,
  1976,
  5274,
  1537,
  1142,
  634,
  1707

In [23]:
train_dataset=train_dataset.remove_columns("process_text")

In [24]:
train_dataset["labels"]

[1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,


In [25]:
from sklearn.model_selection import StratifiedKFold

In [26]:
folds=5
SKF=StratifiedKFold(n_splits=folds,shuffle=True,random_state=26)


In [27]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)


In [28]:
from transformers import RobertaForSequenceClassification

In [29]:
device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [44]:
model=RobertaForSequenceClassification.from_pretrained("roberta-large",num_labels=2)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

In [45]:
from transformers import AdamW
from torch.nn.modules.loss import CrossEntropyLoss
from transformers import get_scheduler
lr_rate=1e-4
warm_rate=0.01
epoch=10
optimizer=AdamW(model.parameters(),lr=lr_rate)

/opt/anaconda3/envs/tweet/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [46]:
err=[]
y_pred_tot=[]
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

In [47]:
split_result=SKF.split(train_dataset,train_dataset["labels"])

In [51]:
from tqdm.auto import tqdm
import evaluate
metrics=evaluate.load("accuracy")
metrics_train=evaluate.load("accuracy")
#train_steps=epoch*len(train_dataloader)*folds
#val_steps=epoch*len(val_dataloader)*folds
#train_bar=tqdm(train_steps)
#val_bar=tqdm(val_steps)
#split_result=SKF.split(train_dataset,train_dataset["labels"])
epoch=3
total_bar=tqdm(range(epoch*folds))
for num_epoch in range(epoch):
    SKF=StratifiedKFold(n_splits=folds,shuffle=True,random_state=(26+num_epoch))
    split_result=SKF.split(train_dataset,train_dataset["labels"])
    for train_index,val_index in split_result:
        loss_total=0
        train_split=train_dataset.select(train_index)
        val_split=train_dataset.select(val_index)
        trainsplit_dataloader=DataLoader(train_split,shuffle=True,batch_size=16,collate_fn=data_collator)
        valsplit_dataloader=DataLoader(val_split,shuffle=True,batch_size=16,collate_fn=data_collator)
        train_steps=len(trainsplit_dataloader)
        lr_scheduler=get_scheduler('linear',optimizer=optimizer,num_warmup_steps=20,num_training_steps=train_steps)
        val_steps=len(valsplit_dataloader)
        train_bar=tqdm(range(train_steps))
        val_bar=tqdm(range(val_steps))
        model.train()
        for batch in trainsplit_dataloader:
            batch={k:v.to(device) for k,v in batch.items()}
            optimizer.zero_grad()
            output=model(**batch)
            loss=output.loss
            loss.backward()
            loss_total+=loss.item()
            optimizer.step()
            lr_scheduler.step()

            train_bar.update(1)
            train_pred=torch.argmax(output.logits,dim=-1)
            metrics_train.add_batch(predictions=train_pred,references=batch["labels"])
        print(metrics_train.compute())
        print(loss_total/len(trainsplit_dataloader))
        err.append(loss_total/len(trainsplit_dataloader))
        model.eval()
        for batch in valsplit_dataloader:
            batch={k:v.to(device) for k,v in batch.items()}
            output=model(**batch)
            logits=output.logits
            preds=torch.argmax(logits,dim=-1)
            metrics.add_batch(predictions=preds,references=batch["labels"])
            val_bar.update(1)
        acc=metrics.compute()
        print(acc)
        y_pred_tot.append(acc)
        total_bar.update(1)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/265 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

{'accuracy': 0.5131236698983211}
0.6960585992291288
{'accuracy': 0.5189035916824196}


  0%|          | 0/265 [00:00<?, ?it/s]

...

  0%|          | 0/67 [00:00<?, ?it/s]

{'accuracy': 0.5011820330969267}
0.6956723678786799
{'accuracy': 0.5193945127719962}


  0%|          | 0/265 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

{'accuracy': 0.5009456264775414}
0.6984963176385411
{'accuracy': 0.5193945127719962}


In [39]:
err

[0.7156842294728981,
 0.7124377655533125,
 0.7067638421958348,
 0.7073782720655765,
 0.702883281797733,
 0.7037922933416546,
 0.7030008612938647,
 0.7023803920116065,
 0.7041094856442146,
 0.6997941167849415,
 0.7013539408737759,
 0.6997006809936379,
 0.7028913295493936,
 0.6994152858572186,
 0.6989995362623682,
 0.7006221442852381,
 0.6998362705392658,
 0.6991544015002701,
 0.7013358721193278,
 0.6953125746744984,
 0.6976327086394688,
 0.6980765898272676,
 0.7001853272599994,
 0.6970657179940422,
 0.6974211409406842,
 0.7001595713057608,
 0.6974165097722468,
 0.6971615390957526,
 0.6957113261492747,
 0.6969738523915129,
 0.6969982732017085,
 0.698897980743984,
 0.6934399973671391,
 0.7006004742856295,
 0.6955197975320636,
 0.695373072939099,
 0.6954101087912073,
 0.6963921956296237,
 0.7003186079691042,
 0.6952548458891095,
 0.6972741579109768,
 0.6964354254164786,
 0.6949837452960465,
 0.6959301127577727,
 0.695606801419888,
 0.6961054118174427,
 0.695156874071877,
 0.695890291007059

In [38]:
y_pred_tot

[{'accuracy': 0.5189035916824196},
 {'accuracy': 0.48015122873345933},
 {'accuracy': 0.4806054872280038},
 {'accuracy': 0.5193945127719962},
 {'accuracy': 0.5193945127719962},
 {'accuracy': 0.5189035916824196},
 {'accuracy': 0.5198487712665406},
 {'accuracy': 0.5193945127719962},
 {'accuracy': 0.5193945127719962},
 {'accuracy': 0.5193945127719962},
 {'accuracy': 0.4810964083175803},
 {'accuracy': 0.5198487712665406},
 {'accuracy': 0.4806054872280038},
 {'accuracy': 0.5193945127719962},
 {'accuracy': 0.4806054872280038},
 {'accuracy': 0.5189035916824196},
 {'accuracy': 0.5198487712665406},
 {'accuracy': 0.5193945127719962},
 {'accuracy': 0.5193945127719962},
 {'accuracy': 0.4806054872280038},
 {'accuracy': 0.5189035916824196},
 {'accuracy': 0.5198487712665406},
 {'accuracy': 0.5193945127719962},
 {'accuracy': 0.5193945127719962},
 {'accuracy': 0.5193945127719962},
 {'accuracy': 0.4810964083175803},
 {'accuracy': 0.5198487712665406},
 {'accuracy': 0.5193945127719962},
 {'accuracy': 0.519

In [ ]:
len(train_dataset)

In [ ]:
len(train_dataloader)

In [ ]:
"""
from tqdm.auto import tqdm
import evaluate
metrics=evaluate.load("accuracy")
model.to(device)
process_bar=tqdm(range(train_steps))
for num in range(epoch):
    model.train()
    loss_total=0
    for batch in train_dataloader:
        batch={k:v.to(device) for k,v in batch.items()}
        output=model(**batch)
        loss=output.loss
        loss.backward()
        loss_total+=loss.item()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        process_bar.update(1)
    print(loss_total)
    model.eval()
"""

In [ ]:
def train_eval(model,optimizer,lr_scheduler,loss_total,process_bar):
    metrics=evaluate.load("accuracy")
    model.to(device)
    process_bar=tqdm(range(train_steps))
    for num in range(epoch):
        model.train()
        loss_total=0
        for batch in train_dataloader:
            batch={k:v.to(device) for k,v in batch.items()}
            output=model(**batch)
            loss=output.loss
            loss.backward()
            loss_total+=loss.item()
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            process_bar.update(1)
        print(loss_total)
        model.eval()
    return ?